In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import pyspark.sql

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [3]:
df_rent = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('rent.csv')

In [4]:
df_rent.count()

10

In [5]:
df_rent.filter(df_rent.rent_space == 'U100501').show()

+----------+----+------+------+
|rent_space|year|quater|   erv|
+----------+----+------+------+
|   U100501|2014|     1|100976|
|   U100501|2014|     2|101140|
|   U100501|2014|     3|101304|
|   U100501|2014|     4|101468|
|   U100501|2015|     1|101633|
+----------+----+------+------+



In [6]:
window_over_erv = Window.partitionBy("erv").orderBy("rent_space")

In [7]:
df_rent.withColumn('index',F.lit(1)).show()

+----------+----+------+------+-----+
|rent_space|year|quater|   erv|index|
+----------+----+------+------+-----+
|   U100501|2014|     1|100976|    1|
|   U100501|2014|     2|101140|    1|
|   U100501|2014|     3|101304|    1|
|   U100501|2014|     4|101468|    1|
|   U100501|2015|     1|101633|    1|
|   U100502|2014|     1| 45364|    1|
|   U100502|2014|     2| 45438|    1|
|   U100502|2014|     3| 45511|    1|
|   U100502|2014|     4| 45585|    1|
|   U100502|2015|     1| 45659|    1|
+----------+----+------+------+-----+



In [8]:
window_over_erv1 = Window.partitionBy("rent_space").orderBy("year")

In [9]:
df_dates = df_rent.withColumn("ERV Growth % Per Quarter",(F.lead("erv").over(window_over_erv1)-df_rent.erv)/df_rent.erv)

In [11]:
df_dates.show(10, truncate=False)

+----------+----+------+------+------------------------+
|rent_space|year|quater|erv   |ERV Growth % Per Quarter|
+----------+----+------+------+------------------------+
|U100501   |2014|1     |100976|0.00162414831247029     |
|U100501   |2014|2     |101140|0.0016215147320545779   |
|U100501   |2014|3     |101304|0.0016188896785911712   |
|U100501   |2014|4     |101468|0.0016261284345803603   |
|U100501   |2015|1     |101633|null                    |
|U100502   |2014|1     |45364 |0.0016312494489022131   |
|U100502   |2014|2     |45438 |0.0016065847968660593   |
|U100502   |2014|3     |45511 |0.0016259805321790337   |
|U100502   |2014|4     |45585 |0.0016233410112975759   |
|U100502   |2015|1     |45659 |null                    |
+----------+----+------+------+------------------------+

